In [99]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader as web
import matplotlib as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np 
import datetime as dt
import yfinance as yf
import pandas_ta 
import warnings
warnings.filterwarnings('ignore')

In [101]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-04-02'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

In [103]:
symbols_list[0:10]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

In [104]:
df = yf.download(tickers = symbols_list,
                 start = start_date,
                 end = end_date).stack()

[*********************100%%**********************]  503 of 503 completed


In [105]:
df

Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2016-04-04 A        37.544666   40.009998   40.520000   39.799999   40.320000   
           AAL      37.726418   39.369999   40.240002   39.150002   39.810001   
           AAPL     25.395544   27.780001   28.047501   27.567499   27.605000   
           ABBV     41.949986   59.209999   59.400002   57.490002   57.639999   
           ABT      36.484840   42.320000   42.669998   42.160000   42.340000   
...                       ...         ...         ...         ...         ...   
2024-04-01 XYL     128.639999  128.639999  129.509995  127.910004  129.250000   
           YUM     140.470001  140.470001  140.729996  138.419998  138.550003   
           ZBH     129.750000  129.750000  131.610001  129.679993  131.610001   
           ZBRA    301.000000  301.000000  304.559998  298.070007  302.589996   
           ZTS     167.020004  167.020004  169.490005  166.119995  168.990005   

Price                   Volume  
Date       Ticker               
2016-04-04 A         2958100.0  
           AAL       7831200.0  
           AAPL    149424800.0  
           ABBV      8108100.0  
           ABT       4407800.0  
...                        ...  
2024-04-01 XYL        716800.0  
           YUM       2227600.0  
           ZBH        745000.0  
           ZBRA       416900.0  
           ZTS       1896500.0  

[991726 rows x 6 columns]

In [126]:
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['High'],
                      low=stock_data['Low'],
                      close=stock_data['Close'],
                      length=14)
     
    if atr is None:
      return np.nan  # Return a single value (e.g., np.nan)
    else:
      return (atr.sub(atr.mean())  # Perform calculations in a single chain
           .div(atr.std()))
 # return (atr - mean) / standard deviation
    
def compute_macd(close):
  """ Wrapper function for pandas_ta.macd handling None values"""
  macd = pandas_ta.macd(close=close, length=20)
  if macd is None:
    # Handle missing data (e.g., return np.nan or a default value)
    return np.nan
  else:
    return macd.sub(macd.mean()).div(macd.std()) # normalised 


In [123]:
def compute_macd(close):
  """ Wrapper function for pandas_ta.macd handling None values"""
  macd = pandas_ta.macd(close=close, length=20)
  if macd is None:
    # Handle missing data (e.g., return np.nan or a default value)
    return np.nan
  else:
    return macd.sub(macd.mean()).div(macd.std())  # Access first column

In [116]:
# def compute_atr(group):
#   """ Wrapper function for pandas_ta.atr handling None values"""
#   atr = pandas_ta.atr(high=group['High'],
#                      low=group['Low'],
#                      close=group['Close'],
#                      length=14)
#   if atr is None:
#     # Return an empty DataFrame (consistent structure)
#     return pd.DataFrame()
#   else:
#     # Return a Series with the calculated ATR value
#     return pd.Series(atr.sub(atr.mean()).div(atr.std()))




In [131]:
# df['garman_klass_vol'] = ((np.log(df['High']) -np.log(df['Low']))**2)/2 - (2*np.log(2)-1)*((np.log(df['Adj Close']) - np.log(df['Open'])))
# df['rsi'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.rsi(close = x, length = 20))

# df['bb_low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: 
#   np.nan if x.empty else (pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 0] 
#                           if pandas_ta.bbands(close=np.log1p(x), length=20) is not None else np.nan)
# )
# df['bb_mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: 
#   np.nan if x.empty else (pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 1] 
#                           if pandas_ta.bbands(close=np.log1p(x), length=20) is not None else np.nan)
# )
# df['bb_high'] = df.groupby(level=1)['Adj Close'].transform(lambda x: 
#   np.nan if x.empty else (pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 2] 
#                           if pandas_ta.bbands(close=np.log1p(x), length=20) is not None else np.nan)
# )

# df['atr'] = df.groupby(level = 1, group_keys=False).apply(compute_atr)

# df['macd'] = df.groupby(level = 1, group_keys=False).apply(compute_macd)

df['dollar_volume'] = (df['Adj Close']*df['Volume'])/1e6 # divide by million 
 



In [132]:
df

Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2016-04-04 A        37.544666   40.009998   40.520000   39.799999   40.320000   
           AAL      37.726418   39.369999   40.240002   39.150002   39.810001   
           AAPL     25.395544   27.780001   28.047501   27.567499   27.605000   
           ABBV     41.949986   59.209999   59.400002   57.490002   57.639999   
           ABT      36.484840   42.320000   42.669998   42.160000   42.340000   
...                       ...         ...         ...         ...         ...   
2024-04-01 XYL     128.639999  128.639999  129.509995  127.910004  129.250000   
           YUM     140.470001  140.470001  140.729996  138.419998  138.550003   
           ZBH     129.750000  129.750000  131.610001  129.679993  131.610001   
           ZBRA    301.000000  301.000000  304.559998  298.070007  302.589996   
           ZTS     167.020004  167.020004  169.490005  166.119995  168.990005   

Price                   Volume  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                       
2016-04-04 A         2958100.0          0.027710        NaN       NaN   
           AAL       7831200.0          0.021143        NaN       NaN   
           AAPL    149424800.0          0.032375        NaN       NaN   
           ABBV      8108100.0          0.123275        NaN       NaN   
           ABT       4407800.0          0.057567        NaN       NaN   
...                        ...               ...        ...       ...   
2024-04-01 XYL        716800.0          0.001905  61.729727  4.843066   
           YUM       2227600.0         -0.005179  62.124773  4.907980   
           ZBH        745000.0          0.005607  58.310060  4.821167   
           ZBRA       416900.0          0.002267  62.308535  5.627400   
           ZTS       1896500.0          0.004731  34.624873  5.091082   

Price                bb_mid   bb_high  macd  dollar_volume  
Date       Ticker                                           
2016-04-04 A            NaN       NaN   NaN     111.060877  
           AAL          NaN       NaN   NaN     295.443121  
           AAPL         NaN       NaN   NaN    3794.724091  
           ABBV         NaN       NaN   NaN     340.134677  
           ABT          NaN       NaN   NaN     160.817879  
...                     ...       ...   ...            ...  
2024-04-01 XYL     4.859430  4.875794   NaN      92.209152  
           YUM     4.933904  4.959829   NaN     312.910975  
           ZBH     4.853251  4.885335   NaN      96.663750  
           ZBRA    5.670821  5.714242   NaN     125.486900  
           ZTS     5.169119  5.247157   NaN     316.753438  

[991726 rows x 13 columns]

In [129]:
df['macd'].info()

<class 'pandas.core.series.Series'>
MultiIndex: 991726 entries, (Timestamp('2016-04-04 00:00:00'), 'A') to (Timestamp('2024-04-01 00:00:00'), 'ZTS')
Series name: macd
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 11.4+ MB


In [51]:
df.groupby(level= 1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20) if not x.empty else np.nan)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [31]:
pandas_ta.bbands(close =df.xs('AAPL', level = 1)['Adj Close'], length=20)

,BBL_20_2.0,BBM_20_2.0,BBU_20_2.0,BBB_20_2.0,BBP_20_2.0
Date,,,,,
2016-04-04,NaN,NaN,NaN,NaN,NaN
2016-04-05,NaN,NaN,NaN,NaN,NaN
2016-04-06,NaN,NaN,NaN,NaN,NaN
2016-04-07,NaN,NaN,NaN,NaN,NaN
2016-04-08,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2024-03-25,165.949335,174.211500,182.473665,9.485212,0.296573
2024-03-26,166.050678,173.565500,181.080322,8.659350,0.243474
2024-03-27,166.565361,173.160000,179.754638,7.616815,0.511373


In [53]:
pandas_ta.bbands(close=np.log1p(df), length=20)

In [133]:
df

Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2016-04-04 A        37.544666   40.009998   40.520000   39.799999   40.320000   
           AAL      37.726418   39.369999   40.240002   39.150002   39.810001   
           AAPL     25.395544   27.780001   28.047501   27.567499   27.605000   
           ABBV     41.949986   59.209999   59.400002   57.490002   57.639999   
           ABT      36.484840   42.320000   42.669998   42.160000   42.340000   
...                       ...         ...         ...         ...         ...   
2024-04-01 XYL     128.639999  128.639999  129.509995  127.910004  129.250000   
           YUM     140.470001  140.470001  140.729996  138.419998  138.550003   
           ZBH     129.750000  129.750000  131.610001  129.679993  131.610001   
           ZBRA    301.000000  301.000000  304.559998  298.070007  302.589996   
           ZTS     167.020004  167.020004  169.490005  166.119995  168.990005   

Price                   Volume  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                       
2016-04-04 A         2958100.0          0.027710        NaN       NaN   
           AAL       7831200.0          0.021143        NaN       NaN   
           AAPL    149424800.0          0.032375        NaN       NaN   
           ABBV      8108100.0          0.123275        NaN       NaN   
           ABT       4407800.0          0.057567        NaN       NaN   
...                        ...               ...        ...       ...   
2024-04-01 XYL        716800.0          0.001905  61.729727  4.843066   
           YUM       2227600.0         -0.005179  62.124773  4.907980   
           ZBH        745000.0          0.005607  58.310060  4.821167   
           ZBRA       416900.0          0.002267  62.308535  5.627400   
           ZTS       1896500.0          0.004731  34.624873  5.091082   

Price                bb_mid   bb_high  macd  dollar_volume  
Date       Ticker                                           
2016-04-04 A            NaN       NaN   NaN     111.060877  
           AAL          NaN       NaN   NaN     295.443121  
           AAPL         NaN       NaN   NaN    3794.724091  
           ABBV         NaN       NaN   NaN     340.134677  
           ABT          NaN       NaN   NaN     160.817879  
...                     ...       ...   ...            ...  
2024-04-01 XYL     4.859430  4.875794   NaN      92.209152  
           YUM     4.933904  4.959829   NaN     312.910975  
           ZBH     4.853251  4.885335   NaN      96.663750  
           ZBRA    5.670821  5.714242   NaN     125.486900  
           ZTS     5.169119  5.247157   NaN     316.753438  

[991726 rows x 13 columns]

### 3. Aggregate to monthly level and filter tp 150 most liquid stocks for each month 
- To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency

In [152]:
last_cols = [c for c in df.columns.unique() if c not in ['dollar_volume', 'Volume', 'Open',
                                                          'High', 'Low', 'Close']]
last_cols

df1 =  df.unstack('Ticker')['dollar_volume'].resample('M').mean().stack('Ticker').to_frame('dollar_volume')
# M is monthly
# We then restack it 

df2 = df.unstack()[last_cols].resample('M').last().stack('Ticker')
data = pd.concat([df1, df2], axis=1) # issue with MACD formula
data

dollar_volume   Adj Close  garman_klass_vol        rsi  \
Date       Ticker                                                           
2016-04-30 A           60.883702   38.398602          0.024752        NaN   
           AAL        359.276042   33.241802          0.029972        NaN   
           AAPL      4039.316503   21.423494          0.035975        NaN   
           ABBV       290.969707   43.633457          0.129744        NaN   
           ABT        318.119176   33.741272          0.070327        NaN   
...                          ...         ...               ...        ...   
2024-04-30 XYL         92.209152  128.639999          0.001905  61.729727   
           YUM        312.910975  140.470001         -0.005179  62.124773   
           ZBH         96.663750  129.750000          0.005607  58.310060   
           ZBRA       125.486900  301.000000          0.002267  62.308535   
           ZTS        316.753438  167.020004          0.004731  34.624873   

                     bb_low    bb_mid   bb_high  macd  
Date       Ticker                                      
2016-04-30 A       3.627733  3.669042  3.710351   NaN  
           AAL     3.549626  3.642540  3.735453   NaN  
           AAPL    3.135048  3.231915  3.328782   NaN  
           ABBV    3.740925  3.778749  3.816573   NaN  
           ABT     3.579932  3.641429  3.702927   NaN  
...                     ...       ...       ...   ...  
2024-04-30 XYL     4.843066  4.859430  4.875794   NaN  
           YUM     4.907980  4.933904  4.959829   NaN  
           ZBH     4.821167  4.853251  4.885335   NaN  
           ZBRA    5.627400  5.670821  5.714242   NaN  
           ZTS     5.091082  5.169119  5.247157   NaN  

[47829 rows x 8 columns]

- calculate 5-year rolling average for dollar volume for each stock 

In [164]:
# data['dollar_volume'] = data['dollar_volume'].unstack('Ticker').rolling(5*12).mean().stack()

# next we want dollar vol rank which is based on dollar volume to give top 150 most liquid stocks 

# data['dollar_vol_rank'] = (data.groupby('Date')['dollar_volume'].rank(ascending=False))

# data = data[data['dollar_vol_rank'] < 150].drop(['dollar_volume', 'dollar_vol_rank'], axis =1)
data

Adj Close  garman_klass_vol        rsi    bb_low  \
Date       Ticker                                                      
2021-03-31 AAL      23.900000          0.002324  60.457977  3.061577   
           AAPL    120.056305          0.005282  46.979648  4.757720   
           ABBV     95.737534          0.042772  54.261934  4.516806   
           ABT     113.700897          0.020929  51.461152  4.698549   
           ACN     264.932404          0.019358  62.283712  5.469067   
...                       ...               ...        ...       ...   
2024-04-30 VRTX    420.480011          0.000083  52.672264  6.011348   
           VZ       42.279999         -0.002472  65.805006  3.686859   
           WFC      57.610001          0.001864  65.031639  4.046765   
           WMT      60.000000          0.002085  57.531770  4.101671   
           XOM     116.989998         -0.001792  74.876325  4.657018   

                     bb_mid   bb_high  macd  
Date       Ticker                            
2021-03-31 AAL     3.178512  3.295447   NaN  
           AAPL    4.790046  4.822372   NaN  
           ABBV    4.555067  4.593328   NaN  
           ABT     4.731431  4.764313   NaN  
           ACN     5.540429  5.611792   NaN  
...                     ...       ...   ...  
2024-04-30 VRTX    6.029728  6.048108   NaN  
           VZ      3.722924  3.758989   NaN  
           WFC     4.064396  4.082027   NaN  
           WMT     4.119864  4.138056   NaN  
           XOM     4.719849  4.782680   NaN  

[5662 rows x 7 columns]